In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import csv
import sys
import requests
import math

In [ ]:
headers = {
    'Content-Type': 'application/json',
    'Origin': 'https://ieeexplore.ieee.org',
}
pageNo = 1
recordIds = []
totalPages = pageNo
while(pageNo<=totalPages):
  data = '''{"action":"search",
  "newsearch":true,
  "matchBoolean":true,
  "queryText":"((\\"All Metadata\\":moon) AND \\"All Metadata\\":mars)",
  "ranges":["1872_2021_Year"],
  "highlight":true,
  "returnFacets":["ALL"],
  "returnType":"SEARCH",
  "matchPubs":true,
  "rowsPerPage":"100",
  "pageNumber":''' + str(pageNo) +'}'

  response = requests.post('https://ieeexplore.ieee.org/rest/search', headers=headers, data=str(data))
  # print(response)
  response = response.json()
  # print(response)
  # print(len(response['records']))
  splitInd = len('/document/')
  totalRecords = response['totalRecords']
  totalPages = math.ceil(totalRecords/100)
  recordsCount = min(totalRecords - (pageNo-1)*100, 100)
  records = response['records']
  print(totalPages, recordsCount)
  print(response)
  for recordInd in range(0, recordsCount):
    # print(records[0]['documentLink'][splitInd:-1])
    # print(recordInd)
    try:
      recordIds.append(records[recordInd]['documentLink'][splitInd:-1])
      # print(recordInd, records[recordInd]['documentLink'][splitInd:-1])
    except Exception as e:
      print(repr(e), 'Document link not available :', recordInd+(pageNo-1)*100+1)
  pageNo+=1
print(len(recordIds))

In [ ]:
print(len(recordIds))

In [ ]:
bibs = []
headers={
    'Accept': 'application/json',
    # 'Content-Type':'application/json'
}
data={
    'recordIds' : None,
    'download-format' : 'download-bibtex',
    'citations-format' :'citation-abstract'
}
url = 'https://ieeexplore.ieee.org/xpl/downloadCitations'
for recordId in recordIds:
  try:
    print(recordId)
    data['recordIds'] = str(recordId)
    response = requests.post(url, headers=headers, data=data) 
    bib = response.text
    # print(bib)
    bib = bib.replace("<br>", "")
    # print(bib)
    bibtex = bib.strip("}").strip("\t ") + '\n'
    # print(bibtex)
    ind = bibtex.index("{")
    bib_dict = {}
    bib_dict["type"] = bibtex[:ind]
    bibtex = bibtex[ind+1:]
    ind = bibtex.index("\n")
    bib_dict["id"] = bibtex[:ind-1]
    bibtex = bibtex[ind+1:]
    while(bibtex != "" and bibtex != "\n"): #len(bibtext) > 3):
      # print(bibtex)
      # print("bib", bibtex, len(bibtex))
      ind = bibtex.index("\n")
      attribute = bibtex[:ind].strip("\t,")
      # print("attr",attribute)
      try:
        ind_attr = attribute.index("=")
      except Exception as e:
        bibtex = bibtex[1:]
        continue
      # print(ind_attr)
      bib_dict[attribute[:ind_attr].strip()] = attribute[ind_attr+1:].strip('{} \r,')
      bibtex = bibtex[ind + 1:]
    # print(bib_dict)
    bibs.append(bib_dict)
  except Exception as e:
    continue

In [ ]:
print(len(bibs), bibs)

In [ ]:
output_json = {}
for bib in range(0, len(bibs)):
  print(bibs[bib])
  output_json[bib] = bibs[bib]

In [ ]:
print(output_json)

In [ ]:
import json
filename = "/content/drive/My Drive/bibtex_ieee.json"    
with open(filename, 'w') as outfile:
    json.dump(output_json, outfile)